In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import librosa
import shutil
import keras


import keras.backend as K
import tensorflow as tf
from scipy import signal

from keras.optimizers import *

from keras.regularizers import *

from keras import regularizers, optimizers
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.utils import *
from keras.callbacks import *
from sklearn.model_selection import *
from keras.preprocessing.image import *
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)
from keras import backend as K
K.get_session().list_devices()

/root/anaconda3/envs/audio/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/root/anaconda3/envs/audio/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 10968078746),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:1, GPU, 10968078746)]

In [2]:
os.listdir('../audio-data/')
train_path = '../audio-data/audio_train/'
test_path = '../audio-data/audio_test/'
train = pd.read_csv('../audio-data/train.csv')
test = pd.read_csv('../audio-data/sample_submission.csv')
# print('training samples: ', len(os.listdir(train_path)))
# print('test samples: ', len(os.listdir(test_path)))
# print('training labels: ', len(train.label.unique()))
# print(train.head())
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index('fname', inplace=True)
test.set_index('fname', inplace=True)
train['label_idx'] = train.label.apply(lambda x: label_idx[x])




In [3]:
class Config(object):
    def __init__(self,
                sampling_rate=44100, audio_duration=2, n_classes=41,
                use_mfcc=False, n_folds=10, learning_rate=0.0001,
                max_epochs=50, n_mfcc=20, use_log_sp=False, use_mixup=False, alpha=0.2, use_log_mel_sp=False):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_log_sp = use_log_sp
        self.use_mfcc = use_mfcc
        self.use_mixup = use_mixup
        self.use_log_mel_sp = use_log_mel_sp
        self.alpha = alpha
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            # np.floor 计算比每一个元素小或相等的最大的整数
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        elif self.use_log_sp:
            self.dim = (self.audio_duration*100-1, self.sampling_rate//100+1, 3)
        elif self.use_log_mel_sp:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)
            






In [4]:
# spectrogram.shape[0] = time*100 - 1
# spectrogram.shape[1] = rate/100 + 1
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data - min_data) / (max_data - min_data + 1e-6)
    return data - 0.5

In [5]:
import numpy as np


class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

In [6]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255):
    def eraser(input_img):
        img_h, img_w, _ = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [7]:
%%time
X_train = np.load('X_44100x5_log_mel_sp_train.npy')
X_test = np.load('X_44100x5_log_mel_sp_test.npy')
# X_train = np.load('X_1d_train.npy')
# X_test = np.load('X_1d_test.npy')
y_train = np.load('y_train.npy')

print(X_train.shape, X_test.shape, y_train.shape)

(9473, 256, 431, 1) (9400, 256, 431, 1) (9473, 41)
CPU times: user 271 ms, sys: 10.8 s, total: 11.1 s
Wall time: 20.4 s


In [8]:
%%time
def normalization(x):
    mean = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    x = (x - mean) / std
    return x
X_train = normalization(X_train)
X_test = normalization(X_test)
print(X_train.shape, X_test.shape)

(9473, 256, 431, 1) (9400, 256, 431, 1)
CPU times: user 20.6 s, sys: 27.2 s, total: 47.8 s
Wall time: 46.1 s


In [19]:
%%time
def mixup(data, targets, alpha):
        size = data.shape[0]
        weight = np.random.beta(alpha, alpha, size)
      
        x_weight = weight.reshape(size, 1, 1, 1)
#         x_weight = weight.reshape(size, 1, 1)
        y_weight = weight.reshape(size, 1)
        index = np.random.permutation(size)
        x1, x2 = data, data[index]
        x = x1 * x_weight + x2 * (1 - x_weight)
        y1, y2 = targets, targets[index]
        y = y1 * y_weight + y2 * (1 - y_weight)
        return x, y
X_train_mixup, y_train_mixup = mixup(X_train, y_train, 2)
print(X_train_mixup.shape, y_train_mixup.shape)

(9473, 256, 431, 1) (9473, 41)
CPU times: user 6.57 s, sys: 13.8 s, total: 20.4 s
Wall time: 20 s


In [9]:
class ParallelModelCheckpoint(ModelCheckpoint):
    def __init__(self,model,filepath, monitor='val_loss', verbose=1,
                 save_best_only=True, save_weights_only=True,
                 mode='auto', period=1):
        self.single_model = model
        super(ParallelModelCheckpoint,self).__init__(filepath, monitor, verbose,save_best_only, save_weights_only,mode, period)

    def set_model(self, model):
        super(ParallelModelCheckpoint,self).set_model(self.single_model)


class CustomModelCheckpoint(keras.callbacks.Callback):

    def __init__(self, model, path):
        self.model = model
        self.path = path
        self.best_loss = np.inf

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        if val_loss < self.best_loss:
            print("\nValidation loss decreased from {} to {}, saving model".format(self.best_loss, val_loss))
            self.model.save_weights(self.path, overwrite=True)
            self.best_loss = val_loss



In [10]:
def get_model(config):
    nclass = config.n_classes
    
    inp = Input(shape=(config.dim[0], config.dim[1], 1))
    x = Convolution2D(32, (3,3), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(32, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(64, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(64, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)

    x = Convolution2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
#     x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(256)(x)
# #     x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
#     x = Activation("relu")(x)
    out = Dense(nclass, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)
    model.summary()
    print(len(model.layers))
    print(config.dim)
    return model

In [10]:
%%time
# log mel sp 44100x5
            
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=7, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5"
CHECKPOINT_FOLDER = 'checkpionts_log_mel_sp_44000x5'



if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 64

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
#     parallel_model = multi_gpu_model(model, gpus=2)
    
    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    
#     adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6, amsgrad=True)
#     model.compile(loss='categorical_crossentropy',
#                        optimizer=adam,
#                          metrics=['acc'])
    sgd = SGD(lr=0.00001, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    #     checkpoint = ModelCheckpoint(CHECKPOINT_FOLDER+'/best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
    checkpoint = CustomModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=10)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks_list = [checkpoint, early, tb, rlrop, log]
    
    history = model.fit(X, y, validation_data=(X_val, y_val), callbacks=[checkpoint], 
                        batch_size=batch_size, epochs=config.max_epochs, shuffle=True)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)


    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)





train shape:  (8108, 256, 431, 1) (8108, 41)
val shape:  (1365, 256, 431, 1) (1365, 41)
Fold:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 431, 32)      128       
_________________________________________________________________
activation_7 (Activation)    (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 215, 32)      9248      
_____________________________________________

Epoch 26/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.2062 - acc: 0.9382 - val_loss: 1.4244 - val_acc: 0.7209
Epoch 27/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.1455 - acc: 0.9597 - val_loss: 1.4868 - val_acc: 0.7055
Epoch 28/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.1625 - acc: 0.9489 - val_loss: 1.4292 - val_acc: 0.6974
Epoch 29/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.0780 - acc: 0.9815 - val_loss: 0.8938 - val_acc: 0.8168

Validation loss decreased from 1.1532698383261433 to 0.8937533621386294, saving model
Epoch 30/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.0481 - acc: 0.9917 - val_loss: 0.8878 - val_acc: 0.8271

Validation loss decreased from 0.8937533621386294 to 0.8878387273012937, saving model
Epoch 31/300
8108/8108 [==============================] - 45s 6ms/step - loss: 0.0426 - acc: 0.9940 - val_loss: 0.8919 - val_acc: 0.8300
Epoc

8112/8112 [==============================] - 44s 5ms/step - loss: 1.3326 - acc: 0.6255 - val_loss: 2.0759 - val_acc: 0.4761

Validation loss decreased from 2.1430226736958633 to 2.075886449946979, saving model
Epoch 7/300
8112/8112 [==============================] - 45s 5ms/step - loss: 1.1948 - acc: 0.6691 - val_loss: 1.8394 - val_acc: 0.5364

Validation loss decreased from 2.075886449946979 to 1.8394100101969857, saving model
Epoch 8/300
8112/8112 [==============================] - 45s 6ms/step - loss: 1.0828 - acc: 0.7023 - val_loss: 1.8321 - val_acc: 0.5415

Validation loss decreased from 1.8394100101969857 to 1.8320850184697774, saving model
Epoch 9/300
8112/8112 [==============================] - 45s 6ms/step - loss: 0.9977 - acc: 0.7173 - val_loss: 1.5925 - val_acc: 0.6047

Validation loss decreased from 1.8320850184697774 to 1.592461321741058, saving model
Epoch 10/300
8112/8112 [==============================] - 46s 6ms/step - loss: 0.8827 - acc: 0.7493 - val_loss: 1.5312 - va

Train on 8115 samples, validate on 1358 samples
Epoch 1/300
8115/8115 [==============================] - 49s 6ms/step - loss: 2.9465 - acc: 0.2067 - val_loss: 4.7848 - val_acc: 0.1119

Validation loss decreased from inf to 4.784766863652989, saving model
Epoch 2/300
8115/8115 [==============================] - 44s 5ms/step - loss: 2.3690 - acc: 0.3373 - val_loss: 3.2228 - val_acc: 0.2121

Validation loss decreased from 4.784766863652989 to 3.2227633540163336, saving model
Epoch 3/300
8115/8115 [==============================] - 45s 6ms/step - loss: 1.9101 - acc: 0.4657 - val_loss: 2.7766 - val_acc: 0.2946

Validation loss decreased from 3.2227633540163336 to 2.7766234607865075, saving model
Epoch 4/300
8115/8115 [==============================] - 45s 6ms/step - loss: 1.6489 - acc: 0.5326 - val_loss: 3.1918 - val_acc: 0.2673
Epoch 5/300
8115/8115 [==============================] - 45s 5ms/step - loss: 1.4325 - acc: 0.6000 - val_loss: 2.4393 - val_acc: 0.4293

Validation loss decreased f

Train on 8121 samples, validate on 1352 samples
Epoch 1/300
8121/8121 [==============================] - 45s 6ms/step - loss: 2.8728 - acc: 0.2177 - val_loss: 5.7302 - val_acc: 0.1013

Validation loss decreased from inf to 5.730176327496591, saving model
Epoch 2/300
8121/8121 [==============================] - 46s 6ms/step - loss: 2.2222 - acc: 0.3761 - val_loss: 2.4785 - val_acc: 0.3291

Validation loss decreased from 5.730176327496591 to 2.4785092088597764, saving model
Epoch 3/300
8121/8121 [==============================] - 43s 5ms/step - loss: 1.8349 - acc: 0.4834 - val_loss: 3.1429 - val_acc: 0.2936
Epoch 4/300
8121/8121 [==============================] - 43s 5ms/step - loss: 1.5716 - acc: 0.5581 - val_loss: 1.9853 - val_acc: 0.4667

Validation loss decreased from 2.4785092088597764 to 1.985259339654234, saving model
Epoch 5/300
8121/8121 [==============================] - 43s 5ms/step - loss: 1.3678 - acc: 0.6193 - val_loss: 1.5234 - val_acc: 0.6021

Validation loss decreased fr

Train on 8125 samples, validate on 1348 samples
Epoch 1/300
8125/8125 [==============================] - 48s 6ms/step - loss: 2.9274 - acc: 0.1985 - val_loss: 3.9202 - val_acc: 0.1142

Validation loss decreased from inf to 3.920248627308922, saving model
Epoch 2/300
8125/8125 [==============================] - 45s 5ms/step - loss: 2.3814 - acc: 0.3328 - val_loss: 7.5827 - val_acc: 0.1001
Epoch 3/300
8125/8125 [==============================] - 46s 6ms/step - loss: 1.9537 - acc: 0.4428 - val_loss: 3.2713 - val_acc: 0.2634

Validation loss decreased from 3.920248627308922 to 3.2712998814681873, saving model
Epoch 4/300
8125/8125 [==============================] - 45s 6ms/step - loss: 1.6383 - acc: 0.5333 - val_loss: 2.3672 - val_acc: 0.3902

Validation loss decreased from 3.2712998814681873 to 2.3671821769691714, saving model
Epoch 5/300
8125/8125 [==============================] - 45s 5ms/step - loss: 1.4418 - acc: 0.5903 - val_loss: 2.3172 - val_acc: 0.4332

Validation loss decreased f

Train on 8146 samples, validate on 1327 samples
Epoch 1/300
8146/8146 [==============================] - 47s 6ms/step - loss: 2.9312 - acc: 0.2098 - val_loss: 5.8837 - val_acc: 0.0686

Validation loss decreased from inf to 5.883746755509423, saving model
Epoch 2/300
8146/8146 [==============================] - 45s 5ms/step - loss: 2.3033 - acc: 0.3499 - val_loss: 2.8950 - val_acc: 0.2690

Validation loss decreased from 5.883746755509423 to 2.8949762623204087, saving model
Epoch 3/300
8146/8146 [==============================] - 45s 6ms/step - loss: 1.9376 - acc: 0.4494 - val_loss: 2.5051 - val_acc: 0.3210

Validation loss decreased from 2.8949762623204087 to 2.505142078780553, saving model
Epoch 4/300
8146/8146 [==============================] - 45s 6ms/step - loss: 1.6946 - acc: 0.5182 - val_loss: 3.5836 - val_acc: 0.2185
Epoch 5/300
8146/8146 [==============================] - 45s 6ms/step - loss: 1.4717 - acc: 0.5811 - val_loss: 1.8077 - val_acc: 0.5041

Validation loss decreased fr

In [11]:
%%time
# log_mel_sp_44100x5 fine tune 
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=7, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5"
CHECKPOINT_FOLDER = 'checkpionts_log_mel_sp_44000x5'



if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 64

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
#     parallel_model = multi_gpu_model(model, gpus=2)
    
    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    
#     adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6, amsgrad=True)
#     model.compile(loss='categorical_crossentropy',
#                        optimizer=adam,
#                          metrics=['acc'])
    sgd = SGD(lr=0.00001, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    #     checkpoint = ModelCheckpoint(CHECKPOINT_FOLDER+'/best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
    checkpoint = CustomModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=10)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks_list = [checkpoint, early, tb, rlrop, log]
    
    history = model.fit(X, y, validation_data=(X_val, y_val), callbacks=[checkpoint], 
                        batch_size=batch_size, epochs=10, shuffle=True)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)


    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)






train shape:  (8108, 256, 431, 1) (8108, 41)
val shape:  (1365, 256, 431, 1) (1365, 41)
Fold:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_49 (Batc (None, 256, 431, 32)      128       
_________________________________________________________________
activation_49 (Activation)   (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 128, 215, 32)      9248      
_____________________________________________

Train on 8111 samples, validate on 1362 samples
Epoch 1/10
8111/8111 [==============================] - 49s 6ms/step - loss: 0.2169 - acc: 0.9502 - val_loss: 0.8602 - val_acc: 0.7680

Validation loss decreased from inf to 0.860225069715413, saving model
Epoch 2/10
8111/8111 [==============================] - 46s 6ms/step - loss: 0.2193 - acc: 0.9495 - val_loss: 0.8610 - val_acc: 0.7687
Epoch 3/10
8111/8111 [==============================] - 47s 6ms/step - loss: 0.2163 - acc: 0.9525 - val_loss: 0.8601 - val_acc: 0.7680

Validation loss decreased from 0.860225069715413 to 0.8601464948640171, saving model
Epoch 4/10
8111/8111 [==============================] - 47s 6ms/step - loss: 0.2147 - acc: 0.9520 - val_loss: 0.8607 - val_acc: 0.7673
Epoch 5/10
8111/8111 [==============================] - 47s 6ms/step - loss: 0.2191 - acc: 0.9506 - val_loss: 0.8603 - val_acc: 0.7680
Epoch 6/10
8111/8111 [==============================] - 47s 6ms/step - loss: 0.2144 - acc: 0.9522 - val_loss: 0.8604 - v

Train on 8115 samples, validate on 1358 samples
Epoch 1/10
8115/8115 [==============================] - 52s 6ms/step - loss: 0.1893 - acc: 0.9591 - val_loss: 0.8115 - val_acc: 0.7865

Validation loss decreased from inf to 0.8114897305905732, saving model
Epoch 2/10
8115/8115 [==============================] - 45s 6ms/step - loss: 0.1893 - acc: 0.9577 - val_loss: 0.8119 - val_acc: 0.7887
Epoch 3/10
8115/8115 [==============================] - 47s 6ms/step - loss: 0.1897 - acc: 0.9602 - val_loss: 0.8117 - val_acc: 0.7894
Epoch 4/10
8115/8115 [==============================] - 47s 6ms/step - loss: 0.1921 - acc: 0.9560 - val_loss: 0.8119 - val_acc: 0.7887
Epoch 5/10
8115/8115 [==============================] - 47s 6ms/step - loss: 0.1928 - acc: 0.9558 - val_loss: 0.8118 - val_acc: 0.7887
Epoch 6/10
8115/8115 [==============================] - 47s 6ms/step - loss: 0.1875 - acc: 0.9596 - val_loss: 0.8119 - val_acc: 0.7872
Epoch 7/10
8115/8115 [==============================] - 47s 6ms/step -

Train on 8125 samples, validate on 1348 samples
Epoch 1/10
8125/8125 [==============================] - 51s 6ms/step - loss: 0.2202 - acc: 0.9493 - val_loss: 0.8678 - val_acc: 0.7908

Validation loss decreased from inf to 0.8677879699702079, saving model
Epoch 2/10
8125/8125 [==============================] - 46s 6ms/step - loss: 0.2191 - acc: 0.9511 - val_loss: 0.8672 - val_acc: 0.7908

Validation loss decreased from 0.8677879699702079 to 0.8671770688334275, saving model
Epoch 3/10
8125/8125 [==============================] - 47s 6ms/step - loss: 0.2197 - acc: 0.9497 - val_loss: 0.8675 - val_acc: 0.7908
Epoch 4/10
8125/8125 [==============================] - 46s 6ms/step - loss: 0.2218 - acc: 0.9478 - val_loss: 0.8670 - val_acc: 0.7908

Validation loss decreased from 0.8671770688334275 to 0.8670146671353178, saving model
Epoch 5/10
8125/8125 [==============================] - 46s 6ms/step - loss: 0.2201 - acc: 0.9487 - val_loss: 0.8668 - val_acc: 0.7908

Validation loss decreased from

In [11]:
%%time

# log mel sp 44100x5 mixup
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=7, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5_mixup"
CHECKPOINT_FOLDER = 'checkpoints_log_mel_sp_44000x5_mixup'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 64

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
    if i < 1:
        continue
    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
#     parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-7, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])

#     sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
#     parallel_model.compile(loss='categorical_crossentropy',
#                   optimizer=sgd,
#                   metrics=['acc'])

    checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks = [checkpoint, early, tb, rlrop, log]
    
    training_generator = MixupGenerator(X, y, batch_size=batch_size, alpha=0.5, datagen=None)()
    model.fit_generator(generator=training_generator,
                        steps_per_epoch=X.shape[0] // batch_size,
                        validation_data=(X_val, y_val),
                        epochs=config.max_epochs, verbose=1,
                        callbacks=callbacks)
    
    
# Fine tune
#     model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5' % i)
#     K.set_value(model.optimizer.lr, 0.00001)
#     parallel_model.fit_generator(mixupgen(),
#                         steps_per_epoch=X_train.shape[0] // batch_size,
#                         epochs=10,
#                         validation_data=test_datagen.flow(X_val, y_val), callbacks=None)
#     parallel_model.save_weights(CHECKPOINT_FOLDER + '/besh_%d.h5' % i)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)

    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)







train shape:  (8111, 256, 431, 1) (8111, 41)
val shape:  (1362, 256, 431, 1) (1362, 41)
Fold:  1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 431, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 215, 32)      9248      
_____________________________________________

126/126 [==============================] - 38s 303ms/step - loss: 1.3452 - acc: 0.8027 - val_loss: 1.2627 - val_acc: 0.6902

Epoch 00021: val_loss did not improve
Epoch 22/300
126/126 [==============================] - 38s 305ms/step - loss: 1.3240 - acc: 0.8116 - val_loss: 1.0699 - val_acc: 0.7276

Epoch 00022: val_loss improved from 1.11288 to 1.06991, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_1.h5
Epoch 23/300
126/126 [==============================] - 38s 303ms/step - loss: 1.2841 - acc: 0.8127 - val_loss: 1.4018 - val_acc: 0.6402

Epoch 00023: val_loss did not improve
Epoch 24/300
126/126 [==============================] - 38s 302ms/step - loss: 1.2818 - acc: 0.8189 - val_loss: 1.2972 - val_acc: 0.6652

Epoch 00024: val_loss did not improve
Epoch 25/300
126/126 [==============================] - 38s 302ms/step - loss: 1.2710 - acc: 0.8259 - val_loss: 1.2742 - val_acc: 0.6725

Epoch 00025: val_loss did not improve
Epoch 26/300
126/126 [==============================

Epoch 1/300
126/126 [==============================] - 38s 304ms/step - loss: 3.2649 - acc: 0.1531 - val_loss: 4.1082 - val_acc: 0.1117

Epoch 00001: val_loss improved from inf to 4.10816, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_2.h5
Epoch 2/300
126/126 [==============================] - 39s 309ms/step - loss: 2.8666 - acc: 0.2639 - val_loss: 3.4295 - val_acc: 0.1940

Epoch 00002: val_loss improved from 4.10816 to 3.42952, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_2.h5
Epoch 3/300
126/126 [==============================] - 38s 301ms/step - loss: 2.5792 - acc: 0.3712 - val_loss: 2.9871 - val_acc: 0.2278

Epoch 00003: val_loss improved from 3.42952 to 2.98713, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_2.h5
Epoch 4/300
126/126 [==============================] - 39s 309ms/step - loss: 2.3804 - acc: 0.4348 - val_loss: 2.9931 - val_acc: 0.2711

Epoch 00004: val_loss did not improve
Epoch 5/300
126/126 [==============================] - 38s 30

Epoch 1/300
126/126 [==============================] - 38s 303ms/step - loss: 3.2468 - acc: 0.1637 - val_loss: 4.1959 - val_acc: 0.0891

Epoch 00001: val_loss improved from inf to 4.19594, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_3.h5
Epoch 2/300
126/126 [==============================] - 38s 300ms/step - loss: 2.8797 - acc: 0.2630 - val_loss: 4.6670 - val_acc: 0.0832

Epoch 00002: val_loss did not improve
Epoch 3/300
126/126 [==============================] - 38s 301ms/step - loss: 2.5631 - acc: 0.3642 - val_loss: 3.3879 - val_acc: 0.2113

Epoch 00003: val_loss improved from 4.19594 to 3.38787, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_3.h5
Epoch 4/300
126/126 [==============================] - 38s 301ms/step - loss: 2.3709 - acc: 0.4433 - val_loss: 2.4575 - val_acc: 0.3373

Epoch 00004: val_loss improved from 3.38787 to 2.45754, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_3.h5
Epoch 5/300
126/126 [==============================] - 38s 30

Epoch 1/300
126/126 [==============================] - 39s 307ms/step - loss: 3.2376 - acc: 0.1576 - val_loss: 3.2409 - val_acc: 0.1575

Epoch 00001: val_loss improved from inf to 3.24089, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_4.h5
Epoch 2/300
126/126 [==============================] - 38s 305ms/step - loss: 2.8684 - acc: 0.2625 - val_loss: 4.6752 - val_acc: 0.2078

Epoch 00002: val_loss did not improve
Epoch 3/300
126/126 [==============================] - 37s 295ms/step - loss: 2.5846 - acc: 0.3684 - val_loss: 2.4643 - val_acc: 0.3536

Epoch 00003: val_loss improved from 3.24089 to 2.46435, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_4.h5
Epoch 4/300
126/126 [==============================] - 38s 299ms/step - loss: 2.3609 - acc: 0.4469 - val_loss: 2.3379 - val_acc: 0.3587

Epoch 00004: val_loss improved from 2.46435 to 2.33790, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_4.h5
Epoch 5/300
126/126 [==============================] - 38s 29

126/126 [==============================] - 37s 295ms/step - loss: 1.1348 - acc: 0.8659 - val_loss: 0.6566 - val_acc: 0.8306

Epoch 00039: val_loss did not improve
Epoch 40/300
126/126 [==============================] - 38s 299ms/step - loss: 1.1298 - acc: 0.8690 - val_loss: 0.6566 - val_acc: 0.8306

Epoch 00040: val_loss did not improve
Epoch 41/300
126/126 [==============================] - 38s 299ms/step - loss: 1.1403 - acc: 0.8625 - val_loss: 0.6595 - val_acc: 0.8262

Epoch 00041: val_loss did not improve
Epoch 42/300
126/126 [==============================] - 37s 292ms/step - loss: 1.1329 - acc: 0.8601 - val_loss: 0.6632 - val_acc: 0.8277

Epoch 00042: val_loss did not improve
Epoch 43/300
126/126 [==============================] - 38s 299ms/step - loss: 1.1353 - acc: 0.8704 - val_loss: 0.6574 - val_acc: 0.8277

Epoch 00043: val_loss did not improve
Epoch 44/300
126/126 [==============================] - 37s 294ms/step - loss: 1.1286 - acc: 0.8621 - val_loss: 0.6594 - val_acc: 0.8

126/126 [==============================] - 38s 299ms/step - loss: 1.0547 - acc: 0.8824 - val_loss: 0.8135 - val_acc: 0.8012

Epoch 00047: val_loss did not improve
Epoch 48/300
126/126 [==============================] - 38s 301ms/step - loss: 1.0441 - acc: 0.8895 - val_loss: 0.8143 - val_acc: 0.7982

Epoch 00048: val_loss did not improve
Epoch 49/300
126/126 [==============================] - 37s 297ms/step - loss: 1.0366 - acc: 0.8926 - val_loss: 0.8132 - val_acc: 0.7982

Epoch 00049: val_loss did not improve
Epoch 50/300
126/126 [==============================] - 38s 300ms/step - loss: 1.0517 - acc: 0.8901 - val_loss: 0.8128 - val_acc: 0.7982

Epoch 00050: val_loss did not improve
Epoch 51/300
126/126 [==============================] - 38s 301ms/step - loss: 1.0603 - acc: 0.8848 - val_loss: 0.8122 - val_acc: 0.7997

Epoch 00051: val_loss did not improve
Epoch 52/300
126/126 [==============================] - 38s 304ms/step - loss: 1.0565 - acc: 0.8817 - val_loss: 0.8123 - val_acc: 0.7

Epoch 1/300
127/127 [==============================] - 38s 302ms/step - loss: 3.2319 - acc: 0.1603 - val_loss: 3.0858 - val_acc: 0.1876

Epoch 00001: val_loss improved from inf to 3.08575, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_6.h5
Epoch 2/300
127/127 [==============================] - 38s 301ms/step - loss: 2.8909 - acc: 0.2582 - val_loss: 3.4558 - val_acc: 0.2163

Epoch 00002: val_loss did not improve
Epoch 3/300
127/127 [==============================] - 38s 302ms/step - loss: 2.6287 - acc: 0.3533 - val_loss: 2.3106 - val_acc: 0.3474

Epoch 00003: val_loss improved from 3.08575 to 2.31058, saving model to checkpoints_log_mel_sp_44000x5_mixup/best_6.h5
Epoch 4/300
127/127 [==============================] - 38s 298ms/step - loss: 2.3641 - acc: 0.4465 - val_loss: 2.4397 - val_acc: 0.3708

Epoch 00004: val_loss did not improve
Epoch 5/300
127/127 [==============================] - 38s 297ms/step - loss: 2.1838 - acc: 0.5145 - val_loss: 2.3465 - val_acc: 0.3775

Epoch

In [13]:
%%time

# log mel sp 44100x5 image_aug
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=10, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5_image_aug_10f"
CHECKPOINT_FOLDER = 'checkpoints_log_mel_sp_44000x5_image_aug_10f'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 142

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):

    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
    parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-7, amsgrad=True)
    parallel_model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])

#     sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
#     parallel_model.compile(loss='categorical_crossentropy',
#                   optimizer=sgd,
#                   metrics=['acc'])

    checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks = [checkpoint, early, tb, rlrop, log]
    
    
    datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    preprocessing_function=get_random_eraser(v_l=np.min(X), v_h=np.max(X)) # RANDOM ERASER
    )
    
    datagen.fit(X)
    training_generator = MixupGenerator(X, y, batch_size=batch_size, alpha=0.5, datagen=datagen)()
    
    parallel_model.fit_generator(generator=training_generator,
                        steps_per_epoch=X.shape[0] // batch_size,
                        validation_data=(X_val, y_val),
                        epochs=config.max_epochs, verbose=1,
                        callbacks=callbacks)
    
    
# Fine tune
#     model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5' % i)
#     K.set_value(model.optimizer.lr, 0.00001)
#     parallel_model.fit_generator(mixupgen(),
#                         steps_per_epoch=X_train.shape[0] // batch_size,
#                         epochs=10,
#                         validation_data=test_datagen.flow(X_val, y_val), callbacks=None)
#     parallel_model.save_weights(CHECKPOINT_FOLDER + '/besh_%d.h5' % i)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    parallel_model = multi_gpu_model(model, gpus=2)

    
    # Save train predictions
    predictions = parallel_model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = parallel_model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)








train shape:  (8518, 256, 431, 1) (8518, 41)
val shape:  (955, 256, 431, 1) (955, 41)
Fold:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_19 (Batc (None, 256, 431, 32)      128       
_________________________________________________________________
activation_19 (Activation)   (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 128, 215, 32)      9248      
_______________________________________________

9400/9400 [==============================] - 23s 2ms/step
train shape:  (8520, 256, 431, 1) (8520, 41)
val shape:  (953, 256, 431, 1) (953, 41)
Fold:  1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_25 (Batc (None, 256, 431, 32)      128       
_________________________________________________________________
activation_25 (Activation)   (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 128, 215, 32)      

60/60 [==============================] - 32s 540ms/step - loss: 1.3517 - acc: 0.8058 - val_loss: 1.1623 - val_acc: 0.7251

Epoch 00020: val_loss improved from 1.16912 to 1.16226, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_1.h5
Epoch 21/300
60/60 [==============================] - 32s 534ms/step - loss: 1.2986 - acc: 0.8227 - val_loss: 0.9992 - val_acc: 0.7587

Epoch 00021: val_loss improved from 1.16226 to 0.99920, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_1.h5
Epoch 22/300
60/60 [==============================] - 32s 539ms/step - loss: 1.2931 - acc: 0.8224 - val_loss: 1.2378 - val_acc: 0.7135

Epoch 00022: val_loss did not improve
Epoch 23/300
60/60 [==============================] - 33s 545ms/step - loss: 1.2591 - acc: 0.8281 - val_loss: 1.1579 - val_acc: 0.7293

Epoch 00023: val_loss did not improve
Epoch 24/300
60/60 [==============================] - 32s 537ms/step - loss: 1.2527 - acc: 0.8327 - val_loss: 1.1432 - val_acc: 0.7240

Epo

Epoch 1/300
60/60 [==============================] - 41s 685ms/step - loss: 3.2779 - acc: 0.1575 - val_loss: 3.7622 - val_acc: 0.1313

Epoch 00001: val_loss improved from inf to 3.76224, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_2.h5
Epoch 2/300
60/60 [==============================] - 30s 506ms/step - loss: 2.9184 - acc: 0.2568 - val_loss: 3.5631 - val_acc: 0.1975

Epoch 00002: val_loss improved from 3.76224 to 3.56314, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_2.h5
Epoch 3/300
60/60 [==============================] - 32s 540ms/step - loss: 2.6295 - acc: 0.3553 - val_loss: 4.3686 - val_acc: 0.1418

Epoch 00003: val_loss did not improve
Epoch 4/300
60/60 [==============================] - 33s 543ms/step - loss: 2.3819 - acc: 0.4465 - val_loss: 3.4032 - val_acc: 0.2521

Epoch 00004: val_loss improved from 3.56314 to 3.40319, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_2.h5
Epoch 5/300
60/60 [==========================

60/60 [==============================] - 32s 538ms/step - loss: 0.9985 - acc: 0.8984 - val_loss: 0.7395 - val_acc: 0.8109

Epoch 00039: val_loss did not improve
Epoch 40/300
60/60 [==============================] - 32s 537ms/step - loss: 0.9785 - acc: 0.9025 - val_loss: 0.7367 - val_acc: 0.8141

Epoch 00040: val_loss did not improve
Epoch 41/300
60/60 [==============================] - 33s 545ms/step - loss: 0.9977 - acc: 0.8968 - val_loss: 0.7509 - val_acc: 0.8067

Epoch 00041: val_loss did not improve
Epoch 42/300
60/60 [==============================] - 33s 544ms/step - loss: 0.9830 - acc: 0.9027 - val_loss: 0.7366 - val_acc: 0.8172

Epoch 00042: val_loss did not improve
Epoch 43/300
60/60 [==============================] - 33s 542ms/step - loss: 0.9756 - acc: 0.9055 - val_loss: 0.7356 - val_acc: 0.8193

Epoch 00043: val_loss did not improve
Epoch 44/300
60/60 [==============================] - 33s 544ms/step - loss: 0.9734 - acc: 0.9014 - val_loss: 0.7358 - val_acc: 0.8193

Epoch 0

...

60/60 [==============================] - 32s 531ms/step - loss: 0.9938 - acc: 0.9012 - val_loss: 0.7482 - val_acc: 0.8087

Epoch 00039: val_loss did not improve
Epoch 40/300
60/60 [==============================] - 32s 538ms/step - loss: 0.9872 - acc: 0.9014 - val_loss: 0.7293 - val_acc: 0.8171

Epoch 00040: val_loss did not improve
Epoch 41/300
60/60 [==============================] - 32s 534ms/step - loss: 0.9896 - acc: 0.9061 - val_loss: 0.7262 - val_acc: 0.8192

Epoch 00041: val_loss did not improve
Epoch 42/300
60/60 [==============================] - 32s 539ms/step - loss: 0.9890 - acc: 0.9025 - val_loss: 0.7234 - val_acc: 0.8182

Epoch 00042: val_loss did not improve
Epoch 43/300
60/60 [==============================] - 32s 539ms/step - loss: 0.9875 - acc: 0.9040 - val_loss: 0.7242 - val_acc: 0.8140

Epoch 00043: val_loss did not improve
9400/9400 [==============================] - 21s 2ms/step
train shape:  (8528, 256, 431, 1) (8528, 41)
val shape:  (945, 256, 431, 1) (945, 41)

60/60 [==============================] - 32s 533ms/step - loss: 0.9982 - acc: 0.8994 - val_loss: 0.6682 - val_acc: 0.8370

Epoch 00060: val_loss did not improve
Epoch 61/300
60/60 [==============================] - 32s 537ms/step - loss: 1.0032 - acc: 0.8934 - val_loss: 0.6684 - val_acc: 0.8381

Epoch 00061: val_loss did not improve
Epoch 62/300
60/60 [==============================] - 32s 535ms/step - loss: 1.0017 - acc: 0.8987 - val_loss: 0.6687 - val_acc: 0.8392

Epoch 00062: val_loss did not improve
Epoch 63/300
60/60 [==============================] - 32s 531ms/step - loss: 0.9974 - acc: 0.9020 - val_loss: 0.6682 - val_acc: 0.8392

Epoch 00063: val_loss did not improve
Epoch 64/300
60/60 [==============================] - 32s 532ms/step - loss: 0.9884 - acc: 0.8962 - val_loss: 0.6688 - val_acc: 0.8392

Epoch 00064: val_loss did not improve
Epoch 65/300
60/60 [==============================] - 32s 535ms/step - loss: 0.9958 - acc: 0.9031 - val_loss: 0.6690 - val_acc: 0.8392

Epoch 0

60/60 [==============================] - 32s 535ms/step - loss: 1.6231 - acc: 0.7089 - val_loss: 1.8342 - val_acc: 0.5217

Epoch 00012: val_loss did not improve
Epoch 13/300
60/60 [==============================] - 32s 533ms/step - loss: 1.5443 - acc: 0.7418 - val_loss: 1.1996 - val_acc: 0.6903

Epoch 00013: val_loss improved from 1.42418 to 1.19956, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_7.h5
Epoch 14/300
60/60 [==============================] - 32s 534ms/step - loss: 1.5210 - acc: 0.7455 - val_loss: 1.1123 - val_acc: 0.7116

Epoch 00014: val_loss improved from 1.19956 to 1.11226, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_7.h5
Epoch 15/300
60/60 [==============================] - 32s 534ms/step - loss: 1.4724 - acc: 0.7665 - val_loss: 1.9904 - val_acc: 0.4867

Epoch 00015: val_loss did not improve
Epoch 16/300
60/60 [==============================] - 32s 536ms/step - loss: 1.4395 - acc: 0.7696 - val_loss: 1.2420 - val_acc: 0.6713

Epo

Epoch 1/300
60/60 [==============================] - 41s 691ms/step - loss: 3.2476 - acc: 0.1601 - val_loss: 4.5841 - val_acc: 0.1348

Epoch 00001: val_loss improved from inf to 4.58406, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_8.h5
Epoch 2/300
60/60 [==============================] - 31s 510ms/step - loss: 2.8303 - acc: 0.2791 - val_loss: 4.1424 - val_acc: 0.2017

Epoch 00002: val_loss improved from 4.58406 to 4.14242, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_8.h5
Epoch 3/300
60/60 [==============================] - 32s 534ms/step - loss: 2.5150 - acc: 0.3938 - val_loss: 3.1363 - val_acc: 0.3079

Epoch 00003: val_loss improved from 4.14242 to 3.13627, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_8.h5
Epoch 4/300
60/60 [==============================] - 32s 526ms/step - loss: 2.3542 - acc: 0.4494 - val_loss: 2.1012 - val_acc: 0.4586

Epoch 00004: val_loss improved from 3.13627 to 2.10118, saving model to checkpoints

60/60 [==============================] - 33s 543ms/step - loss: 1.0281 - acc: 0.9000 - val_loss: 0.8006 - val_acc: 0.7994

Epoch 00040: val_loss improved from 0.80124 to 0.80057, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_8.h5
Epoch 41/300
60/60 [==============================] - 32s 541ms/step - loss: 1.0097 - acc: 0.8937 - val_loss: 0.7950 - val_acc: 0.8025

Epoch 00041: val_loss improved from 0.80057 to 0.79503, saving model to checkpoints_log_mel_sp_44000x5_image_aug_10f/best_8.h5
Epoch 42/300
60/60 [==============================] - 32s 541ms/step - loss: 1.0193 - acc: 0.8989 - val_loss: 0.7981 - val_acc: 0.8068

Epoch 00042: val_loss did not improve
Epoch 43/300
60/60 [==============================] - 33s 542ms/step - loss: 1.0320 - acc: 0.8904 - val_loss: 0.7980 - val_acc: 0.8047

Epoch 00043: val_loss did not improve
Epoch 44/300
60/60 [==============================] - 32s 538ms/step - loss: 1.0107 - acc: 0.8986 - val_loss: 0.7964 - val_acc: 0.8025

Epo

In [ ]:
%%time

# log mel sp 44100x5 image_aug + class_weights
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=7, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5_image_aug_10f"
CHECKPOINT_FOLDER = 'checkpoints_log_mel_sp_44000x5_image_aug_10f'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 142

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):

    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
    parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-7, amsgrad=True)
    parallel_model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])

#     sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
#     parallel_model.compile(loss='categorical_crossentropy',
#                   optimizer=sgd,
#                   metrics=['acc'])

    checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks = [checkpoint, early, tb, rlrop, log]
    
    
    datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    preprocessing_function=get_random_eraser(v_l=np.min(X), v_h=np.max(X)) # RANDOM ERASER
    )
    
    datagen.fit(X)
    training_generator = MixupGenerator(X, y, batch_size=batch_size, alpha=0.5, datagen=datagen)()
    
    parallel_model.fit_generator(generator=training_generator,
                        steps_per_epoch=X.shape[0] // batch_size,
                        validation_data=(X_val, y_val),
                        epochs=config.max_epochs, verbose=1,
                        callbacks=callbacks)
    
    
# Fine tune
#     model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5' % i)
#     K.set_value(model.optimizer.lr, 0.00001)
#     parallel_model.fit_generator(mixupgen(),
#                         steps_per_epoch=X_train.shape[0] // batch_size,
#                         epochs=10,
#                         validation_data=test_datagen.flow(X_val, y_val), callbacks=None)
#     parallel_model.save_weights(CHECKPOINT_FOLDER + '/besh_%d.h5' % i)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    parallel_model = multi_gpu_model(model, gpus=2)

    
    # Save train predictions
    predictions = parallel_model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = parallel_model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)









In [22]:
def get_model(config):
    nclass = config.n_classes
    
    inp = Input(shape=(config.dim[0], config.dim[1], 1))
    x = Convolution2D(32, (3,3), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(32, (3,3), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(64, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(64, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(128, (1,1), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)
    
    x = Convolution2D(256, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(256, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(256, (1,1), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPool2D(strides=(2, 2))(x)

    x = Convolution2D(512, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(512, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Convolution2D(512, (1,1), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
#     x = MaxPool2D(strides=(2, 2))(x)
    

#     x = GlobalAveragePooling2D()(x)
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
#     x = Activation("relu")(x)
    out = Dense(nclass, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)
    model.summary()
    print(len(model.layers))
    print(config.dim)
    return model

In [16]:
%%time

# log mel sp 44100x5 image_aug_m2
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=7, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5_image_aug_m2"
CHECKPOINT_FOLDER = 'checkpoints_log_mel_sp_44000x5_image_aug_m2'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 40

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):

    X = X_train[train_split]
    y = y_train[train_split]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
#     parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-7, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])

#     sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
#     parallel_model.compile(loss='categorical_crossentropy',
#                   optimizer=sgd,
#                   metrics=['acc'])

    checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks = [checkpoint, early, tb, rlrop, log]
    
    
    datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    preprocessing_function=get_random_eraser(v_l=np.min(X), v_h=np.max(X)) # RANDOM ERASER
    )
    
    datagen.fit(X)
    training_generator = MixupGenerator(X, y, batch_size=batch_size, alpha=0.5, datagen=datagen)()
    
    model.fit_generator(generator=training_generator,
                        steps_per_epoch=X.shape[0] // batch_size,
                        validation_data=(X_val, y_val),
                        epochs=config.max_epochs, verbose=1,
                        callbacks=callbacks)
    
    
# Fine tune
#     model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5' % i)
#     K.set_value(model.optimizer.lr, 0.00001)
#     parallel_model.fit_generator(mixupgen(),
#                         steps_per_epoch=X_train.shape[0] // batch_size,
#                         epochs=10,
#                         validation_data=test_datagen.flow(X_val, y_val), callbacks=None)
#     parallel_model.save_weights(CHECKPOINT_FOLDER + '/besh_%d.h5' % i)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)

    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)









train shape:  (8108, 256, 431, 1) (8108, 41)
val shape:  (1365, 256, 431, 1) (1365, 41)
Fold:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_86 (Batc (None, 256, 431, 32)      128       
_________________________________________________________________
activation_86 (Activation)   (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 128, 215, 64)      18496     
_____________________________________________

Epoch 1/300
202/202 [==============================] - 80s 398ms/step - loss: 3.4728 - acc: 0.1661 - val_loss: 7.3321 - val_acc: 0.0844

Epoch 00001: val_loss improved from inf to 7.33205, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_1.h5
Epoch 2/300
202/202 [==============================] - 79s 390ms/step - loss: 2.6859 - acc: 0.3486 - val_loss: 2.4343 - val_acc: 0.3436

Epoch 00002: val_loss improved from 7.33205 to 2.43428, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_1.h5
Epoch 3/300
202/202 [==============================] - 79s 389ms/step - loss: 2.3532 - acc: 0.4509 - val_loss: 2.6182 - val_acc: 0.3524

Epoch 00003: val_loss did not improve
Epoch 4/300
202/202 [==============================] - 79s 389ms/step - loss: 2.1541 - acc: 0.5235 - val_loss: 2.0386 - val_acc: 0.4295

Epoch 00004: val_loss improved from 2.43428 to 2.03855, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_1.h5
Epoch 5/300
202/202 [===================

202/202 [==============================] - 79s 390ms/step - loss: 2.1788 - acc: 0.5113 - val_loss: 2.4282 - val_acc: 0.3630

Epoch 00004: val_loss improved from 3.08419 to 2.42817, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_2.h5
Epoch 5/300
202/202 [==============================] - 79s 390ms/step - loss: 2.0391 - acc: 0.5610 - val_loss: 1.8387 - val_acc: 0.5349

Epoch 00005: val_loss improved from 2.42817 to 1.83870, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_2.h5
Epoch 6/300
202/202 [==============================] - 79s 390ms/step - loss: 1.9391 - acc: 0.5954 - val_loss: 1.7955 - val_acc: 0.5033

Epoch 00006: val_loss improved from 1.83870 to 1.79552, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_2.h5
Epoch 7/300
202/202 [==============================] - 79s 390ms/step - loss: 1.8456 - acc: 0.6223 - val_loss: 1.7652 - val_acc: 0.5298

Epoch 00007: val_loss improved from 1.79552 to 1.76521, saving model to checkpoints_lo


Epoch 00042: val_loss did not improve
Epoch 43/300
202/202 [==============================] - 79s 390ms/step - loss: 0.9297 - acc: 0.8996 - val_loss: 0.7493 - val_acc: 0.8090

Epoch 00043: val_loss did not improve
Epoch 44/300
202/202 [==============================] - 79s 390ms/step - loss: 0.9357 - acc: 0.8978 - val_loss: 0.7369 - val_acc: 0.8082

Epoch 00044: val_loss did not improve
Epoch 45/300
202/202 [==============================] - 79s 390ms/step - loss: 0.9268 - acc: 0.8989 - val_loss: 0.7395 - val_acc: 0.8185

Epoch 00045: val_loss did not improve
Epoch 46/300
202/202 [==============================] - 79s 389ms/step - loss: 0.9194 - acc: 0.9012 - val_loss: 0.7469 - val_acc: 0.8090

Epoch 00046: val_loss did not improve
Epoch 47/300
202/202 [==============================] - 79s 389ms/step - loss: 0.9276 - acc: 0.9001 - val_loss: 0.7394 - val_acc: 0.8134

Epoch 00047: val_loss did not improve
Epoch 48/300
202/202 [==============================] - 79s 390ms/step - loss: 0.

202/202 [==============================] - 79s 390ms/step - loss: 2.0869 - acc: 0.5521 - val_loss: 2.2563 - val_acc: 0.4212

Epoch 00004: val_loss improved from 2.99879 to 2.25631, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_3.h5
Epoch 5/300
202/202 [==============================] - 79s 390ms/step - loss: 1.9642 - acc: 0.5900 - val_loss: 2.8949 - val_acc: 0.3041

Epoch 00005: val_loss did not improve
Epoch 6/300
202/202 [==============================] - 79s 391ms/step - loss: 1.8650 - acc: 0.6220 - val_loss: 1.8633 - val_acc: 0.4897

Epoch 00006: val_loss improved from 2.25631 to 1.86334, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_3.h5
Epoch 7/300
202/202 [==============================] - 79s 390ms/step - loss: 1.8021 - acc: 0.6449 - val_loss: 2.4676 - val_acc: 0.3962

Epoch 00007: val_loss did not improve
Epoch 8/300
202/202 [==============================] - 79s 390ms/step - loss: 1.7068 - acc: 0.6720 - val_loss: 2.1780 - val_acc: 0.4588



Epoch 00005: val_loss did not improve
Epoch 6/300
203/203 [==============================] - 79s 388ms/step - loss: 1.9969 - acc: 0.5800 - val_loss: 2.0162 - val_acc: 0.4608

Epoch 00006: val_loss did not improve
Epoch 7/300
203/203 [==============================] - 79s 388ms/step - loss: 1.9128 - acc: 0.6025 - val_loss: 1.4171 - val_acc: 0.6146

Epoch 00007: val_loss improved from 1.95076 to 1.41707, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_4.h5
Epoch 8/300
203/203 [==============================] - 79s 389ms/step - loss: 1.8106 - acc: 0.6414 - val_loss: 1.8780 - val_acc: 0.4926

Epoch 00008: val_loss did not improve
Epoch 9/300
203/203 [==============================] - 79s 388ms/step - loss: 1.8062 - acc: 0.6361 - val_loss: 1.7678 - val_acc: 0.5237

Epoch 00009: val_loss did not improve
Epoch 10/300
203/203 [==============================] - 79s 389ms/step - loss: 1.7191 - acc: 0.6712 - val_loss: 1.3293 - val_acc: 0.6206

Epoch 00010: val_loss improved from

Epoch 1/300
203/203 [==============================] - 82s 402ms/step - loss: 3.2850 - acc: 0.2117 - val_loss: 3.1896 - val_acc: 0.2411

Epoch 00001: val_loss improved from inf to 3.18958, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_5.h5
Epoch 2/300
203/203 [==============================] - 79s 390ms/step - loss: 2.5230 - acc: 0.4012 - val_loss: 2.8600 - val_acc: 0.2663

Epoch 00002: val_loss improved from 3.18958 to 2.85997, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_5.h5
Epoch 3/300
203/203 [==============================] - 79s 389ms/step - loss: 2.2125 - acc: 0.4982 - val_loss: 2.8941 - val_acc: 0.3798

Epoch 00003: val_loss did not improve
Epoch 4/300
203/203 [==============================] - 79s 390ms/step - loss: 2.0643 - acc: 0.5459 - val_loss: 2.1640 - val_acc: 0.4585

Epoch 00004: val_loss improved from 2.85997 to 2.16405, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_5.h5
Epoch 5/300
203/203 [===================

203/203 [==============================] - 79s 390ms/step - loss: 1.0466 - acc: 0.8725 - val_loss: 0.8175 - val_acc: 0.7960

Epoch 00039: val_loss did not improve
Epoch 40/300
203/203 [==============================] - 79s 391ms/step - loss: 1.0642 - acc: 0.8724 - val_loss: 0.8178 - val_acc: 0.8019

Epoch 00040: val_loss did not improve
Epoch 41/300
203/203 [==============================] - 79s 391ms/step - loss: 1.0312 - acc: 0.8828 - val_loss: 0.8184 - val_acc: 0.7975

Epoch 00041: val_loss did not improve
Epoch 42/300
203/203 [==============================] - 79s 390ms/step - loss: 1.0648 - acc: 0.8728 - val_loss: 0.8157 - val_acc: 0.7982

Epoch 00042: val_loss did not improve
Epoch 43/300
203/203 [==============================] - 79s 390ms/step - loss: 1.0675 - acc: 0.8735 - val_loss: 0.8163 - val_acc: 0.7967

Epoch 00043: val_loss did not improve
Epoch 44/300
203/203 [==============================] - 79s 391ms/step - loss: 1.0715 - acc: 0.8762 - val_loss: 0.8164 - val_acc: 0.7


Epoch 00002: val_loss improved from 4.08391 to 3.08936, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_6.h5
Epoch 3/300
203/203 [==============================] - 79s 391ms/step - loss: 2.2317 - acc: 0.4957 - val_loss: 2.1913 - val_acc: 0.3821

Epoch 00003: val_loss improved from 3.08936 to 2.19133, saving model to checkpoints_log_mel_sp_44000x5_image_aug_m2/best_6.h5
Epoch 4/300
203/203 [==============================] - 79s 390ms/step - loss: 2.0728 - acc: 0.5502 - val_loss: 3.0257 - val_acc: 0.3157

Epoch 00004: val_loss did not improve
Epoch 5/300
203/203 [==============================] - 79s 390ms/step - loss: 1.9892 - acc: 0.5840 - val_loss: 3.2135 - val_acc: 0.3060

Epoch 00005: val_loss did not improve
Epoch 6/300
203/203 [==============================] - 79s 389ms/step - loss: 1.8658 - acc: 0.6186 - val_loss: 2.5581 - val_acc: 0.4265

Epoch 00006: val_loss did not improve
Epoch 7/300
203/203 [==============================] - 79s 390ms/step - loss: 1.7999 

In [25]:
%%time
# log_mel_sp_44100x5 
config = Config(sampling_rate=44100, audio_duration=5, n_classes=41, use_log_mel_sp=True, n_folds=5, max_epochs=300, n_mfcc=128*2)
PREDICTION_FOLDER = "predictions_log_mel_sp_44000x5_mixup_2x"
CHECKPOINT_FOLDER = 'checkpionts_log_mel_sp_44000x5_mixup_2x'



if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 64

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
    X = np.r_[X_train[train_split], X_train_mixup[train_split]]
    y = np.r_[y_train[train_split], y_train_mixup[train_split]]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = X_train[train_split]
#     y = y_train[train_split]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model(config)
#     parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])
#     sgd = SGD(lr=0.00001, momentum=0.9, nesterov=True)
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=sgd,
#                   metrics=['accuracy'])

    #     checkpoint = ModelCheckpoint(CHECKPOINT_FOLDER+'/best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
    checkpoint = CustomModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks_list = [checkpoint, early, tb, rlrop, log]
    
    history = model.fit(X, y, validation_data=(X_val, y_val), callbacks=callbacks_list, 
                        batch_size=batch_size, epochs=config.max_epochs, shuffle=True)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)


    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)







train shape:  (15144, 256, 431, 1) (15144, 41)
val shape:  (1901, 256, 431, 1) (1901, 41)
Fold:  0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 256, 431, 1)       0         
_________________________________________________________________
conv2d_180 (Conv2D)          (None, 256, 431, 32)      320       
_________________________________________________________________
batch_normalization_188 (Bat (None, 256, 431, 32)      128       
_________________________________________________________________
activation_180 (Activation)  (None, 256, 431, 32)      0         
_________________________________________________________________
max_pooling2d_106 (MaxPoolin (None, 128, 215, 32)      0         
_________________________________________________________________
conv2d_181 (Conv2D)          (None, 128, 215, 32)      9248      
___________________________________________

15150/15150 [==============================] - 73s 5ms/step - loss: 1.5504 - acc: 0.6754 - val_loss: 1.5835 - val_acc: 0.5769
Epoch 7/300
15150/15150 [==============================] - 74s 5ms/step - loss: 1.4452 - acc: 0.7081 - val_loss: 1.3217 - val_acc: 0.6607

Validation loss decreased from 1.5492449226319098 to 1.3216732721559867, saving model
Epoch 8/300
15150/15150 [==============================] - 73s 5ms/step - loss: 1.3543 - acc: 0.7369 - val_loss: 1.2418 - val_acc: 0.6812

Validation loss decreased from 1.3216732721559867 to 1.2418281610572302, saving model
Epoch 9/300
15150/15150 [==============================] - 73s 5ms/step - loss: 1.2686 - acc: 0.7626 - val_loss: 1.1657 - val_acc: 0.6791

Validation loss decreased from 1.2418281610572302 to 1.165733650311027, saving model
Epoch 10/300
15150/15150 [==============================] - 73s 5ms/step - loss: 1.1985 - acc: 0.7848 - val_loss: 1.0782 - val_acc: 0.7165

Validation loss decreased from 1.165733650311027 to 1.078181

Train on 15156 samples, validate on 1895 samples
Epoch 1/300
15156/15156 [==============================] - 83s 5ms/step - loss: 3.0668 - acc: 0.1929 - val_loss: 2.9440 - val_acc: 0.2354

Validation loss decreased from inf to 2.9440285369399986, saving model
Epoch 2/300
15156/15156 [==============================] - 73s 5ms/step - loss: 2.4704 - acc: 0.3715 - val_loss: 2.2410 - val_acc: 0.3873

Validation loss decreased from 2.9440285369399986 to 2.2410008869573748, saving model
Epoch 3/300
15156/15156 [==============================] - 73s 5ms/step - loss: 2.0996 - acc: 0.4951 - val_loss: 2.1552 - val_acc: 0.4354

Validation loss decreased from 2.2410008869573748 to 2.1552185105145134, saving model
Epoch 4/300
15156/15156 [==============================] - 73s 5ms/step - loss: 1.8502 - acc: 0.5788 - val_loss: 1.8200 - val_acc: 0.5193

Validation loss decreased from 2.1552185105145134 to 1.8199748279551402, saving model
Epoch 5/300
15156/15156 [==============================] - 73s 5ms

15160/15160 [==============================] - 73s 5ms/step - loss: 0.5856 - acc: 0.9163 - val_loss: 0.5486 - val_acc: 0.8590

Validation loss decreased from 0.8790385199802235 to 0.5486323537488998, saving model
Epoch 24/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5517 - acc: 0.9227 - val_loss: 0.5506 - val_acc: 0.8590
Epoch 25/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5387 - acc: 0.9255 - val_loss: 0.5586 - val_acc: 0.8637
Epoch 26/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5324 - acc: 0.9250 - val_loss: 0.5513 - val_acc: 0.8621
Epoch 27/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5249 - acc: 0.9246 - val_loss: 0.5512 - val_acc: 0.8611
Epoch 28/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5172 - acc: 0.9255 - val_loss: 0.5543 - val_acc: 0.8627
Epoch 29/300
15160/15160 [==============================] - 73s 5ms/step - loss: 0.5121 - ac